### imports

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Activation, Input, Concatenate, GlobalAveragePooling2D, Reshape, Dense, Multiply
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Model
import os
import cv2
from tensorflow.keras.optimizers import Adam
from keras.models import load_model
from tensorflow.keras.utils import plot_model
import config
import json
import numpy as np
import optuna
from segmentation_models.metrics import iou_score, IOUScore, FScore
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import Callback
from tensorflow.keras.regularizers import l1, l2
from math import ceil

# TF CONFIGURATION
print("Número de GPUs disponíveis: ", len(
    tf.config.list_physical_devices('GPU')))
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)
if gpus:
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU') #força usar a 0
        tf.config.experimental.set_memory_growth(gpus[0], True) #so usa memoria a medida q precisa
    except RuntimeError as e:
        print(e)

2024-02-25 12:22:49.806025: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-25 12:22:49.933269: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-25 12:22:50.446146: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-25 12:22:50.446192: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

Segmentation Models: using `keras` framework.
Número de GPUs disponíveis:  2
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]


2024-02-25 12:22:51.199587: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-25 12:22:51.200400: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-25 12:22:51.209273: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-25 12:22:51.210135: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-25 12:22:51.210933: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from S

### dataset -> (leitura - pré_processamento - generator)

In [2]:
#TRAIN - caminho das pastas
train_images_path = "Dataset/train/Images_Cropped"
train_masks_path = "Dataset/train/Masks_Cropped"
#VAL
val_images_path = "Dataset/val/Images_Cropped"
val_masks_path = "Dataset/val/Masks_Cropped"
#TEST
test_images_path = "Dataset/test/Images_Cropped"
test_masks_path = "Dataset/test/Masks_Cropped"

#TRAIN - lista com todos os nomes dos arquivos de imagem
train_images_list = os.listdir(train_images_path)
train_masks_list = os.listdir(train_masks_path)
#VAL
val_images_list = os.listdir(val_images_path)
val_masks_list = os.listdir(val_masks_path)
#TEST
test_images_list = os.listdir(test_images_path)
test_masks_list = os.listdir(test_masks_path)

#TRAIN - ordenação dos nomes dos arquivos
train_images_list.sort()
train_masks_list.sort()
#VAL
val_images_list.sort()
val_masks_list.sort()
#VAL
test_images_list.sort()
test_masks_list.sort()

In [3]:
def image_mask_generator(images_list, images_path, masks_path, batch_size= config.BATCH_SIZE, output_size=config.IMAGE_SIZE):
    num_samples = len(images_list)
    while True:
        # Embaralhar os índices para garantir que os dados sejam apresentados de forma aleatória
        indices = np.random.permutation(num_samples)
        for i in range(0, num_samples, batch_size):
            batch_indices = indices[i:i+batch_size]
            batch_images = []
            
            batch_masks = []
            for idx in batch_indices:
                img_filename = os.path.join(images_path, images_list[idx])
                mask_filename = os.path.join(masks_path, images_list[idx][:-4] + ".png")

                if not os.path.isfile(img_filename) or not os.path.isfile(mask_filename):
                    print(f"Arquivo de imagem ou máscara não encontrado: {images_list[idx]}")
                    continue

                img = cv2.imread(img_filename)
                mask = cv2.imread(mask_filename, cv2.IMREAD_UNCHANGED) #cv2.IMREAD_GRAYSCALE
                
                if img is None or mask is None:
                    print(f"Falha ao carregar imagem ou máscara: {images_list[idx]}")
                    continue

                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, output_size)
                mask = cv2.resize(mask, output_size, interpolation=cv2.INTER_NEAREST)
                
                img = img.astype(np.float32) / 255.0
                mask = tf.one_hot(mask, 3)

                batch_images.append(img)
                batch_masks.append(mask)

            yield np.array(batch_images), np.array(batch_masks)

In [4]:
#TRAIN
gerador_treino = image_mask_generator(train_images_list, train_images_path, train_masks_path)
#VAL
gerador_validacao = image_mask_generator(val_images_list, val_images_path, val_masks_path)
#TEST
gerador_teste = image_mask_generator(test_images_list, test_images_path, test_masks_path)

### construção do modelo Unet, com backbone EfficientNet e blocos Squeeze-Excitation

In [5]:
""" Função Squeeze Excitation """
# def SqueezeAndExcitation(inputs, ratio=8):
#   b, h, w, c = inputs.shape

#   ## Squeeze
#   x = GlobalAveragePooling2D()(inputs)

#   ## Excitation
#   x = Dense(c//ratio, activation = 'relu', use_bias=False)(x)
#   x = Dense(c, activation = 'sigmoid', use_bias=False)(x)

#   ## Scaling
#   x = inputs * x

#   return x


def SqueezeAndExcitation(inputs, ratio=8):
    b, h, w, c = inputs.shape

    ## Squeeze
    x = GlobalAveragePooling2D()(inputs)

    ## Excitation
    x = Dense(c//ratio, activation='relu', use_bias=False, kernel_regularizer=l2(0.01))(x)
    x = Dense(c, activation='sigmoid', use_bias=False, kernel_regularizer=l2(0.01))(x)

    ## Ensure x has dimensions (batch_size, 1, 1, channels)
    x = tf.expand_dims(x, axis=1)
    x = tf.expand_dims(x, axis=1)

    ## Scaling
    x = inputs * x

    return x


def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding='same', kernel_regularizer=l2(0.01))(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding='same', kernel_regularizer=l2(0.01))(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2,2), strides=2, padding='same', kernel_regularizer=l2(0.01))(inputs)
    # Applying SE block
    x = SqueezeAndExcitation(x)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

def build_efficient_unet(input_shape):
    """ Input """
    inputs = Input(input_shape)

    """ Pre-trained Encoder """
    encoder = EfficientNetB0(include_top=False, weights="imagenet", input_tensor=inputs)
    encoder.summary()

    s1 = encoder.get_layer("input_1").output                          ## 256
    s2 = encoder.get_layer("block2a_expand_activation").output       ## 128
    s3 = encoder.get_layer("block3a_expand_activation").output        ## 64
    s4 = encoder.get_layer("block4a_expand_activation").output        ## 32

    """ Bottleneck """
    b1 = encoder.get_layer("block6a_expand_activation").output        ## 16

    """ Decoder """
    d1 = decoder_block(b1, s4, 512)                                    ## 32
    d2 = decoder_block(d1, s3, 256)                                    ## 64
    d3 = decoder_block(d2, s2, 128)                                    ## 128
    d4 = decoder_block(d3, s1, 64)                                     ## 265

    """ Output """
    outputs = Conv2D(3, 1, padding="same", activation='softmax')(d4)

    model = Model(inputs, outputs, name="EfficientNetB0_UNET")
    
    return model

In [ ]:
modelo = build_efficient_unet(input_shape=(224,224,3))
modelo.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= config.LEARNING_RATE),
              loss='categorical_crossentropy',
              metrics= [IOUScore(class_indexes=1, threshold=0.5, name='iou_disco'), IOUScore(class_indexes=2, threshold=0.5, name='iou_cup'), FScore(class_indexes=1, threshold=0.5, name='dice_disco'), FScore(class_indexes=2, threshold=0.5, name='dice_cup')  ])

In [8]:
# pretrain model decoder
callbacks_list = [EarlyStopping(monitor = 'val_iou_cup', patience = 25, mode = 'auto', verbose = 1), ModelCheckpoint(f"/home/arthur_guilherme/pibic_mack-24/segmentation_refuge/checkpoint/best_model_weights.h5", monitor = 'val_iou_cup', verbose = 1, save_best_only = True,save_weights_only = True, mode= 'max', initial_value_threshold=0.8)]


modelo.fit(gerador_treino, 
           validation_data = gerador_validacao,
           epochs= config.EPOCHS,
           callbacks = callbacks_list,
           steps_per_epoch= ceil(len(train_images_list)/config.BATCH_SIZE),
           validation_steps= ceil(len(val_images_list)/config.BATCH_SIZE),)



Epoch 1/2


2024-02-25 12:22:59.636039: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8700
2024-02-25 12:23:00.012549: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


50/50 [==============================] - ETA: 0s - loss: 31.4655 - iou_disco: 0.5708 - iou_cup: 0.3648 - dice_disco: 0.7093 - dice_cup: 0.5284
Epoch 1: val_iou_cup did not improve from 0.80000
50/50 [==============================] - 20s 244ms/step - loss: 31.4655 - iou_disco: 0.5708 - iou_cup: 0.3648 - dice_disco: 0.7093 - dice_cup: 0.5284 - val_loss: 26.1199 - val_iou_disco: 4.9774e-11 - val_iou_cup: 1.5099e-10 - val_dice_disco: 4.9774e-11 - val_dice_cup: 1.5099e-10
Epoch 2/2
50/50 [==============================] - ETA: 0s - loss: 21.3250 - iou_disco: 0.6983 - iou_cup: 0.4922 - dice_disco: 0.8217 - dice_cup: 0.6570
Epoch 2: val_iou_cup did not improve from 0.80000
50/50 [==============================] - 11s 225ms/step - loss: 21.3250 - iou_disco: 0.6983 - iou_cup: 0.4922 - dice_disco: 0.8217 - dice_cup: 0.6570 - val_loss: 18.2217 - val_iou_disco: 0.5025 - val_iou_cup: 1.5133e-10 - val_dice_disco: 0.6685 - val_dice_cup: 1.5133e-10


In [ ]:
def dice_coefficient(y_true, y_pred, smooth=1):
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + smooth) / (union + smooth)
    return dice
# Definindo a métrica Dice
def dice_metric(y_true, y_pred):
    return dice_coefficient(y_true, y_pred)


In [9]:

# if __name__ == "__main__":
#   input_shape = ((224, 2, 3))
#   modelo = build_efficient_unet(input_shape)
#   modelo.summary()


In [ ]:
def evaluation(model, test_generator, num_batches):
    # Inicializa as listas para armazenar as métricas de cada batch
    iou_scores = []
    dice_scores = []
    accuracies = []
    recalls = []
    f1_scores = []

    # Itera sobre os batches do gerador até alcançar o número desejado de batches
    for _ in range(num_batches):
        # Obtém o próximo batch de dados do gerador
        batch_images, batch_labels = next(test_generator)
        
        # Faz previsões usando o modelo no batch atual
        predicts = model.predict(batch_images)
        
        # Converte as previsões em máscaras binárias
        pred_masks = np.argmax(predicts, axis=-1)
        
        # Converte as máscaras verdadeiras para o formato binário
        true_masks = np.argmax(batch_labels, axis=-1)
        
        # Calcula métricas para o batch atual
        intersection = np.logical_and(true_masks, pred_masks)
        union = np.logical_or(true_masks, pred_masks)
        iou_score = np.sum(intersection) / np.sum(union)
        
        dice_score = (2 * np.sum(intersection)) / (np.sum(true_masks) + np.sum(pred_masks))
        
        accuracy = sklearn.metrics.accuracy_score(true_masks.flatten(), pred_masks.flatten())
        recall = sklearn.metrics.recall_score(true_masks.flatten(), pred_masks.flatten(), average='weighted')
        f1_score = sklearn.metrics.f1_score(true_masks.flatten(), pred_masks.flatten(), average='weighted')
        
        # Armazena as métricas do batch atual
        iou_scores.append(iou_score)
        dice_scores.append(dice_score)
        accuracies.append(accuracy)
        recalls.append(recall)
        f1_scores.append(f1_score)
        
    # Calcula as métricas médias para os batches processados
    mean_iou = np.mean(iou_scores)
    mean_dice = np.mean(dice_scores)
    mean_accuracy = np.mean(accuracies)
    mean_recall = np.mean(recalls)
    mean_f1_score = np.mean(f1_scores)
    
    # Imprime as métricas médias
    print(f"Mean IoU (Jaccard Index): {mean_iou}")
    print(f"Mean Dice Coefficient: {mean_dice}")
    print(f"Mean Accuracy: {mean_accuracy}")
    print(f"Mean Recall: {mean_recall}")
    print(f"Mean F1-score: {mean_f1_score}")
    
    return mean_iou, mean_dice, mean_accuracy, mean_recall, mean_f1_score

evaluation(modelo, gerador_treino, num_batches=16)